In [ ]:
pip install google-play-scraper

In [ ]:
pip install --upgrade google-play-scraper

In [ ]:
pip install play_scraper

In [ ]:
conda info --envs

In [ ]:
import csv
from google_play_scraper import app
from time import sleep
from datetime import datetime, timedelta

# replace with the path to your CSV file
csv_file_path = 'input/apps_sample.csv'

# read the list of app names from the CSV file
app_names = []
with open(csv_file_path, 'r') as csv_file:
    reader = csv.reader(csv_file)
    next(reader) # skip header row
    for row in reader:
        app_name = row[0]
        app_names.append(app_name)

# fetch app information and write to a new CSV file
with open('output/app_info.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['App Name', 'Description', 'Downloads', 'Rating', 'Category', 'Size', 'User_base_size', 'Developer Apps'])

    retries = 3
    rate_limit_refresh_time = None
    for app_name in app_names:
        if rate_limit_refresh_time is not None and rate_limit_refresh_time > datetime.now():
            # wait until rate limit refreshes
            time_to_wait = rate_limit_refresh_time - datetime.now()
            sleep(time_to_wait.seconds + 10) # add 10 seconds buffer

        app_info = None
        retry_count = 0
        while app_info is None:
            try:
                results = app(app_name, lang='en', country='in', results=1)
                if results:
                    app_info = results[0]
                else:
                    print(f"App '{app_name}' not found in Play Store anymore")
            except Exception as e:
                if retry_count < retries:
                    retry_count += 1
                    print(f"Error fetching '{app_name}' app info, retrying ({retry_count}/{retries})...")
                    sleep(5)
                else:
                    raise e
            else:
                retry_count = 0

        if app_info:
            app_id = app_info['app_id']
            app_name = app_info['title']
            description = app_info['description']
            downloads = app_info['installs']
            rating = app_info['score']
            category = app_info['genre']
            size = app_info['size']
            user_base_size = downloads
            developer_apps = app_info['developer_apps']

            writer.writerow([app_name, description, downloads, rating, category, size, user_base_size, developer_apps])


In [ ]:
import csv
from google_play_scraper import app, Sort, reviews, reviews_all
import string

# read app names from input csv file
app_names = []
with open('input/apps_sample.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        app_names.append(row[0])

valid_chars = string.ascii_letters + string.digits + " "
app_names = [app_name for app_name in app_names if all(c in valid_chars for c in app_name)]

# result = app('com.google.android.youtube', lang='en', country='us')
app_names = ['org.minddiagnostics']

# set up output file and write header row
with open('output/app_info.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['App Name', 'Description', 'Number of Downloads', 'App Rating', 'App Category', 'App Size', 'User Base Size', 'Number of Apps Offered'])

    for app_name in app_names:
        # search for the app and retrieve its ID
        search_results = app(app_name, lang='en', country='us')
        if search_results:
            app_id = search_results[0]['app_id']
        else:
            print(f"App '{app_name}' not found in Play Store anymore")
            continue
        
        # fetch the desired information using the app ID
        app_info = app(app_id, lang='en', country='us')
        app_desc = app_info['description']
        app_downloads = app_info['installs']
        app_rating = app_info['score']
        app_category = app_info['category']
        app_size = app_info['size']
        user_base_size = app_info['reviews']
        num_apps = app_info['developer']['apps']
        
        writer.writerow([app_name, app_desc, app_downloads, app_rating, app_category, app_size, user_base_size, num_apps])


In [ ]:
from google_play_scraper import app,Sort,search
import pandas as pd
import numpy as np
import json,os,uuid

In [ ]:
gplay = search(
    "Automotive",
    n_hits=2,
    country= "In",
    lang="en",
)

In [ ]:
g_df = pd.DataFrame(np.array(gplay),columns=['review'])
g_df2 = g_df.join(pd.DataFrame(g_df.pop('review').tolist()))

g_df2 = g_df2.rename(columns={'appId': 'App_URL','title':'App_Name','genre':'Categories','score':'Rating','currency':'Loaction','installs':'Downloads'})
# g_df2 = g_df2.drop(columns={'icon' ,'screenshots','video','videoImage','descriptionHTML','description'})
g_df2

In [2]:
from google_play_scraper import app, collection

# Define the collection to be 'topselling_free' or 'topselling_paid'
collection_name = collection('topselling_free', 'IN', 'apps')

# Define a list of categories you're interested in
categories = ['AUTO_AND_VEHICLES', 'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION', 'EDUCATION',
              'ENTERTAINMENT', 'FINANCE', 'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 'LIBRARIES_AND_DEMO',
              'LIFESTYLE', 'MAPS_AND_NAVIGATION', 'MEDICAL', 'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES', 'PARENTING',
              'PERSONALIZATION', 'PHOTOGRAPHY', 'PRODUCTIVITY', 'SHOPPING', 'SOCIAL', 'SPORTS', 'TOOLS', 'TRAVEL_AND_LOCAL',
              'VIDEO_PLAYERS', 'WEATHER']

top_apps = []
for category in categories:
    search_results = collection_name.get(category=category, num=1)
    if search_results:
        top_app = search_results[0]['appId']
        app_details = app(top_app)
        top_apps.append(app_details)

# Print the results
for app_details in top_apps:
    print(app_details['title'], app_details['score'])


ImportError: cannot import name 'collection' from 'google_play_scraper' (/Users/jahnavik/opt/anaconda3/lib/python3.9/site-packages/google_play_scraper/__init__.py)